## ANNLogitLogit (uses Keras)

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = '' 
file = 'data.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 968 TOTAL PEAKS: 189
Done!


### 3. Extract X & Y

In [3]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)   

### 4. Hyperparameter optimisation  (24 cores -> 1.15 hr)

In [4]:
# param_dict
lr = [0.0001, 0.001, 0.01, 0.1, 1]
neuron = [1, 2, 3, 4, 5]
param_dict = dict(learning_rate=lr, n_neurons=neuron, momentum=0.5, decay=0.0, epochs=400, loss='binary_crossentropy', nesterov=False, verbose=0) 

# Initalise
cv = cb.cross_val.kfold(model=cb.model.NN_LogitLogit,                      
                                X=XTknn,                                 
                                Y=Y,                              
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=40,
                                n_cores=-1) # All the cores (using joblib)                               

# Run and plot
cv.run()  

Number of cores set to: 24
Running ...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:08:23<00:00,  3.74s/it]


Time taken: 73.89 minutes with 24 cores
Done!


In [5]:
# Plot
cv.plot(metric='auc', ci=95, scale=1)



Loading BokehJS ...